In [1]:
pacman::p_load(ggplot2,dplyr,lubridate,ggridges,patchwork,data.table,rain,tidyverse)

In [2]:
## FUNCTIONS ##

process_sub_eukulele<-function(eukulele_dir,label_out,all_results,salmon_dir,salmon_format="num") {
    curr_dir = file.path(eukulele_dir,"taxonomy_estimation")
    for (curr_file in list.files(curr_dir)) {
        eukulele_estimates = read.csv(file.path(curr_dir, curr_file), sep = "\t") %>%
                dplyr::mutate(full_classification=case_when(!is.na(full_classification)~full_classification,
                                                            TRUE~"Unclassified")) %>%
                tidyr::separate(full_classification, sep = ";",
                                                     into = c("Domain","Supergroup","Phylum","Class","Order",
                                                              "Family","Genus","Species"))%>% 
                dplyr::mutate(Domain = unlist(lapply(as.character(EUKulele_out$Domain),trimws)))%>% 
                dplyr::mutate(Supergroup = unlist(lapply(as.character(EUKulele_out$Supergroup),trimws)))%>% 
                dplyr::mutate(Phylum = unlist(lapply(as.character(EUKulele_out$Phylum),trimws)))%>% 
                dplyr::mutate(Class = unlist(lapply(as.character(EUKulele_out$Class),trimws)))%>% 
                dplyr::mutate(Order = unlist(lapply(as.character(EUKulele_out$Order),trimws)))%>% 
                dplyr::mutate(Family = unlist(lapply(as.character(EUKulele_out$Family),trimws)))%>% 
                dplyr::mutate(Genus = unlist(lapply(as.character(EUKulele_out$Genus),trimws)))%>% 
                dplyr::mutate(Species = unlist(lapply(as.character(EUKulele_out$Species),trimws)))
        number_file = unlist(strsplit(curr_file,"_"))[1]
        salmon_file = read.csv(file.path(salmon_dir,paste0(salmon_format,as.character(number_file), "_quant"),
                                 "quant.sf"), sep = "\t")
        matched_file= salmon_file %>% dplyr::full_join(eukulele_estimates,by=c("Name"="transcript_name"))
        matched_file["Sample"] = curr_file
        matched_file["SplitSamp"] = number_file
        matched_file["Type"] = label_out

        matched_file =  matched_file %>%
            tidyr::replace_na(list("Domain"="Uncertain",
                                   "Supergroup"="Uncertain",
                                   "Phylum"="Uncertain",
                                   "Class"="Uncertain",
                                   "Order"="Uncertain",
                                   "Family"="Uncertain",
                                   "Genus"="Uncertain",
                                   "Species"="Uncertain"))
        all_results = all_results %>% dplyr::bind_rows(matched_file)
    }
    return(all_results)
}

## Read in EUKulele results from eukrhythmic output to create functional profile

In [ ]:
eukulele_path=file.path("/vortexfs1/omics/alexander/data/Hu-2022-ALOHA-SPOT/Hu_et_al_2022_eukrhythmic/intermediate-files/04-compare/CAG_eukulele")
salmon_dir="/vortexfs1/omics/alexander/data/Hu-2022-ALOHA-SPOT/Hu_et_al_2022_eukrhythmic/intermediate-files/04-compare/09-CAG-mapping/salmon"
suffixes=list.files(file.path(eukulele_path))
EUKulele_out=data.frame()
EUKulele_out_salmon=data.frame()
for (curr2 in list.files(file.path(eukulele_path,"taxonomy_estimation"))) {
    short_code = unlist(strsplit(curr2,"_CAG"))[1]
    EUKulele_out_curr = read.csv(file.path(eukulele_path,"taxonomy_estimation",curr2), sep = "\t")
    EUKulele_out = EUKulele_out %>% 
                     dplyr::bind_rows(EUKulele_out_curr %>%
                                      separate(full_classification, sep = ";",
                                 c("Domain","Supergroup","Phylum","Class",
                                   "Order","Family","Genus","Species")) %>%
                                     dplyr::mutate(Sample = curr2))
    salmon_input = data.frame(fread(file.path(salmon_dir,paste0(short_code,"_quant"),"quant.sf"),sep="\t"))
    EUKulele_salmon = EUKulele_out %>% tidyr::separate(transcript_name,sep="\\.p",
                                                       into=c("transcript_name","protein_id")) %>%
        dplyr::right_join(salmon_input,by=c("transcript_name"="Name"))
    EUKulele_out_salmon = EUKulele_out_salmon %>% dplyr::bind_rows(EUKulele_salmon)
    print(head(EUKulele_salmon))
}

EUKulele_out[is.na(EUKulele_out)] = "Unclassified"

Warning message:
“Expected 8 pieces. Missing pieces filled with `NA` in 846010 rows [2, 3, 4, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 20, 21, 22, 23, 24, ...].”
Warning message:
“Expected 2 pieces. Missing pieces filled with `NA` in 938236 rows [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, ...].”


  X                 transcript_name protein_id classification_level    Domain
1 0 megahit_SRR11178166_k51_1000060       <NA>              species Eukaryota
2 0 megahit_SRR11178166_k51_1000072       <NA>                class Eukaryota
3 0 megahit_SRR11178166_k51_1000211       <NA>               family Eukaryota
4 0 megahit_SRR11178166_k51_1000372       <NA>               family Eukaryota
5 0 megahit_SRR11178166_k51_1000458       <NA>              species Eukaryota
6 0 megahit_SRR11178166_k51_1000535       <NA>                order Eukaryota
       Supergroup          Phylum             Class              Order
1  Archaeplastida     Chlorophyta   Mamiellophyceae        Mamiellales
2       Alveolata  Dinoflagellata       Dinophyceae               <NA>
3  Archaeplastida     Chlorophyta  Trebouxiophyceae       Chlorellales
4   Stramenopiles      Ochrophyta   Bacillariophyta  Bacillariophyta_X
5       Alveolata  Dinoflagellata       Dinophyceae         Suessiales
6       Alveolata  Dinoflage

Warning message:
“Expected 8 pieces. Missing pieces filled with `NA` in 515473 rows [1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, ...].”
Warning message:
“Expected 2 pieces. Missing pieces filled with `NA` in 938236 rows [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, ...].”


  X                 transcript_name protein_id classification_level    Domain
1 0      megahit_SRR11178167_k51_10          2                genus Eukaryota
2 0    megahit_SRR11178167_k51_1000          1                genus Eukaryota
3 0   megahit_SRR11178167_k51_10000          1                genus Eukaryota
4 0 megahit_SRR11178167_k51_1000014          2               family Eukaryota
5 0 megahit_SRR11178167_k51_1000025          3               family Eukaryota
6 0 megahit_SRR11178167_k51_1000039          1              species Eukaryota
      Supergroup          Phylum            Class              Order
1      Alveolata  Dinoflagellata      Dinophyceae      Gymnodiniales
2  Stramenopiles      Ochrophyta  Bacillariophyta  Bacillariophyta_X
3  Stramenopiles      Ochrophyta  Bacillariophyta  Bacillariophyta_X
4      Alveolata  Dinoflagellata      Dinophyceae      Gonyaulacales
5      Alveolata  Dinoflagellata      Dinophyceae      Gymnodiniales
6       Hacrobia     Cryptophyta    Cryp

Warning message:
“Expected 8 pieces. Missing pieces filled with `NA` in 1305903 rows [1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 21, 22, ...].”
Warning message:
“Expected 2 pieces. Missing pieces filled with `NA` in 2370533 rows [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, ...].”


...

Warning message:
“Expected 8 pieces. Missing pieces filled with `NA` in 1025132 rows [1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 15, 16, 17, 18, 19, 20, 21, 22, ...].”
Warning message:
“Expected 2 pieces. Missing pieces filled with `NA` in 2370533 rows [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, ...].”


  X                 transcript_name protein_id classification_level    Domain
1 0       megahit_SRR11178177_k51_0          1               family Eukaryota
2 0 megahit_SRR11178177_k51_1000014          1              species Eukaryota
3 0 megahit_SRR11178177_k51_1000018          2                class Eukaryota
4 0 megahit_SRR11178177_k51_1000020          1                order Eukaryota
5 0 megahit_SRR11178177_k51_1000025          1                genus Eukaryota
6 0  megahit_SRR11178177_k51_100003          1               family Eukaryota
       Supergroup          Phylum                 Class              Order
1       Alveolata      Ciliophora          Spirotrichea     Choreotrichida
2   Stramenopiles      Ochrophyta       Bacillariophyta  Bacillariophyta_X
3       Alveolata  Dinoflagellata           Dinophyceae               <NA>
4  Archaeplastida     Glaucophyta    Glaucocystophyceae  Glaucocystophytes
5       Alveolata      Ciliophora          Spirotrichea     Choreotrichida
6   

Warning message:
“Expected 8 pieces. Missing pieces filled with `NA` in 874269 rows [1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, ...].”
Warning message:
“Expected 2 pieces. Missing pieces filled with `NA` in 2370533 rows [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, ...].”


  X                 transcript_name protein_id classification_level    Domain
1 0       megahit_SRR11178178_k51_0          1                order Eukaryota
2 0     megahit_SRR11178178_k51_100          1               family Eukaryota
3 0 megahit_SRR11178178_k51_1000000          2                genus Eukaryota
4 0 megahit_SRR11178178_k51_1000005          2               family Eukaryota
5 0 megahit_SRR11178178_k51_1000009          1                genus Eukaryota
6 0 megahit_SRR11178178_k51_1000013          1              species Eukaryota
      Supergroup            Phylum              Class              Order
1   Opisthokonta  Choanoflagellida  Choanoflagellatea       Acanthoecida
2      Alveolata    Dinoflagellata        Dinophyceae      Gonyaulacales
3   Opisthokonta  Choanoflagellida  Choanoflagellatea       Acanthoecida
4      Alveolata        Ciliophora       Spirotrichea        Hypotrichia
5  Stramenopiles        Ochrophyta    Bacillariophyta  Bacillariophyta_X
6       Hacrobia

Warning message:
“Expected 8 pieces. Missing pieces filled with `NA` in 1418358 rows [1, 2, 3, 4, 5, 6, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, ...].”


## Plot first pass at the level of Phylum

In [ ]:
colors_out = c("#853951",
"#66df5d",
"#c73cbf",
"#78bd1a",
"#f18aff",
"#417d00",
"#b987ff",
"#ffac26",
"#4199ff",
"#cb6800",
"#008de2",
"#a89400",
"#3c4d97",
"#fcb972",
"#93246c",
"#ffb29a",
"#bd0060",
"#faaeec",
"#af0012",
"#ff8dae")
total_tally=EUKulele_out %>% 
    dplyr::group_by(Phylum,Sample) %>% dplyr::tally() %>%
    dplyr::rename("TotalN"="n")

ggplot(EUKulele_out %>% 
       dplyr::left_join(total_tally) %>%
       dplyr::mutate(Phylum = dplyr::case_when(TotalN > 10000 ~ Phylum,
                                              TRUE ~ "Other" )) %>% dplyr::group_by(Phylum,Sample) %>% 
       dplyr::tally()) + 
    geom_bar(aes(x = Sample, y = n, fill = Phylum),
             position="stack",stat="identity",color="black") +
    scale_fill_manual(values=colors_out) + ylab("Number of predicted proteins") + 
    theme_bw(base_size=14)+ 
    theme(axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1))